In [1]:
#import csv
import datetime as dt
import pandas as pd
import re

#!python --version    #Python 3.8.2
#csv.__version__      #1.0
#pd.__version__       #1.0.3
#re.__version__       #2.2.1
# datetime standard module

In [2]:
#CUSTOM SETTINGS: set these as appropriate for your environment

# Enter the path to the local data files:
path_datafiles = "OriginalData/"

# Enter the path to save the clean, merged data:
path_mergedfiles = "MergedData/"

# This is the LESO_file from Check_DISP_AllStatesAndTerritories.ipynb
# Please check the file there before trying this notebook/
#LESOfile_all = "DISP_AllStatesAndTerritories_06302020.xlsx"
LESOfile_all = "DISP_AllStatesAndTerritories_03312020.xlsx"
# like "DISP_Shipments_Cancellations_mmddyyyy_mmddyyyy.xlsx"
#LESOfile_qtr = "DISP_Shipments_Cancellations_04012020_06302020.xlsx"
LESOfile_qtr = "DISP_Shipments_Cancellations_01012020_03312020.xlsx"

In [3]:
# used by all dataframes
ordered_columns_list = ['OriginatingFile', 'StateAbbreviation', 'RequestingAgency',
                        'ItemDescription', 'RecordDate', 'AcquisitionValue', 'Quantity',
                        'UnitIncrement', 'AgencyType', 'Item_FSG', 'Item_FSC', 'Item_CC',
                        'Item_Code', 'Justification', 'NSN', 'FSC', 'NIIN', 'DEMILCode',
                        'DEMILIC', 'StationType', 'RequisitionID' ,'CancelledBy',
                        'RTDRef', 'ReasonCancelled']
# used by all dataframes
def get_agency_type(check_data):
    agency_dictionary = {' POLICE':'police',' POL':'police',' PD':'police','POLICIA':'police',
                         ' CONSTABLE':'police',' CSO':'community outreach',' ACADEMY':'training',
                         ' SAFETY':'public safety',' DPS':'public safety',' PSD':'public safety',
                         ' ENFORCEMENT':'law enforcement',' LAW ENF':'law enforcement',
                         ' SHERIFF':'sheriff',' SHERIFF\'S':'sheriff',' SHERIFFS':'sheriff',' SO':'sheriff',
                         ' MARSHALL':'marshall','MARSHAL':'marshall',' PATROL':'patrol',' FIRE':'fire',
                         ' ATTORNEY':'district attorney',' ATTY':'district attorney',' DA':'district attorney',
                         ' PROSECUTER':'district attorney',' PROSECUTOR':'district attorney',
                         ' NATURAL':'nat resources',' CONSERVATION':'nat resources',' PARKS':'nat resources',
                         ' CORRECTIONS':'prison',' CORRECTION':'prison',' PRISON':'prison',
                         ' DETENTION':'prison',' DTF':'drug',
                         ' AGRICULTURE':'nat resources','GAME AND FISH':'nat resources','DNR ':'nat resources',
                         'FISH AND WILDLIFE':'nat resources'}
    for k in agency_dictionary:
        if re.search(k,check_data):
            return agency_dictionary[k]

###### PLEASE RUN Check_DISP_AllStatesAndTerritories.ipynb and Check_DISP_Shipments_Cancellations.ipynb FIRST

That notebook works from assumptions in the check notebooks. For more information about various columns and values:   
https://www.dla.mil/Portals/104/Documents/DispositionServices/LESO/DISP_QuickStartGuide_11012017_hyperlinked.pdf   
https://www.dla.mil/DispositionServices/Offers/Reutilization/LawEnforcement/ProgramFAQs.aspx

This notebook merges the cumulative data and the quarterly data into a single dataframe. The idea is that the original data can be created from the merged dataframe. For analysis inside a notebook, all of these columns would not be needed. This notebook can serve as a model or starting point for analyzing the data across LESO files.   
There are notes at the end for saving the data by quarter in tab-separated files. Might want to look into using pyarrow feather format:
https://arrow.apache.org/docs/python/feather.html

### Columns in Merged Dataframe:

__OriginatingFile__: created from file name and sheet <br>
> TYPE:str LENGTH: varies CHARACTER_SET: [A-Za-z0-9] and []   

__StateAbbreviation__: (maps to 'State') two digit postal abbreviation <br>
> TYPE:str LENGTH: 2 CHARACTER_SET: [A-Z]   

__RequestingAgency__: (maps to 'Station Name (LEA)') where LEA stands for 'Law Enforcement Activity/Agency'<br>
> TYPE:str LENGTH: varies CHARACTER_SET: varies   

__ItemDescription__: (maps to 'Item Name') descriptive item name<br>
> TYPE:str LENGTH: varies CHARACTER_SET: varies   

__RecordDate__: (maps to 'Ship Date' AllStatesAndTerritories file)<br>
> TYPE:datetime64 LENGTH:29 CHARACTER_SET: yyyy-mm-ddT00:00:00.000000000  

__AcquisitionValue__: (maps to 'Acquisition Value') value of the requested items in dollars<br>
> TYPE:float64 LENGTH: varies CHARACTER_SET: [0-9.]  

__Quantity__: (maps to 'Quantity') number of items requested<br>
> TYPE:int LENGTH: varies CHARACTER_SET: [0-9]   

 __UnitIncrement__: (maps to 'UI') unit increment<br>
> TYPE:str LENGTH: varies CHARACTER_SET: varies   

__AgencyType__: infered from 'Station Name (LEA)'<br>
> TYPE:str LENGTH: varies CHARACTER_SET: see Expected Values, can be changed     

__The following four columns are all part of the NATO Stock Number, a 13-digit alphanumeric string__<br>
format {aa: FSG bb: FSC cc:CountryCode dddeeee: non-standard item code}<br>
https://en.wikipedia.org/wiki/NATO_Stock_Number (NOTE: FSCG+NIIN == NSN)<br>
__Item_FSG__: Federal Supply Group number<br>
> TYPE:str LENGTH:2 CHARACTER_SET: alphanumeric   

__Item_FSC__: Federal Supply Classification<br>
> TYPE:str LENGTH:2 CHARACTER_SET: alphanumeric   

__Item_CC__: country of origin from National Codification Bureau (aka NCB)<br>
> TYPE:str LENGTH:2 CHARACTER_SET: alphanumeric   

__Item_Code__: non-standard item code<br>
> TYPE:str LENGTH:7 CHARACTER_SET: alphanumeric   

__The following columns are found only in DISP_AllStatesAndTerritories__<br>
__NSN__ NATO Stock Number; https://en.wikipedia.org/wiki/NATO_Stock_Number (aka: FSCG+NIIN == NSN)<br>
> TYPE:str LENGTH:13 CHARACTER_SET: varies {aa: FSG bb: FSC cc:NCB ddd-eeee: non-standard item code}  

__DEMILCode__ (maps to 'DEMIL Code' AllStatesAndTerritories only) demilitarization code<br>
see https://www.dla.mil/HQ/LogisticsOperations/Services/FIC/DEMILCoding/DEMILCodes/<br>
> TYPE:char LENGTH:1 CHARACTER_SET: [GPFDCEBQA]   

__DEMILIC__ (maps to 'DEMIL IC' AllStatesAndTerritories only) demilitarization intgrity code<br>
https://www.dla.mil/HQ/LogisticsOperations/Services/FIC/DEMILCoding/DEMILCodes/<br>
> TYPE:int LENGTH: varies CHARACTER_SET: [0-9] & 'blank'(means not coded yet)   

__StationType__ (maps to 'StationType' AllStatesAndTerritories only) governmental unit owns the agency<br>
> TYPE:str LENGTH:? CHARACTER_SET: 'State'   

__The following columns are found only in DISP_Shipments_Cancellations__<br>
__FSC__: Federal Supply Classification Group number<br>
NATO Stock Number; https://en.wikipedia.org/wiki/NATO_Stock_Number (aka: FSCG+NIIN == NSN)<br>
> TYPE:str LENGTH:4 CHARACTER_SET: [0-9] varies {aa: FSG bb: FSC cc:NCB ddd-eeee: non-standard item code}  

__NIIN__: National Item Identification number<br>
NATO Stock Number; https://en.wikipedia.org/wiki/NATO_Stock_Number (aka: FSCG+NIIN == NSN)<br>
> TYPE:str LENGTH:9 CHARACTER_SET: varies {aa: FSG bb: FSC cc:NCB ddd-eeee: non-standard item code}   

__Justification__: (maps to 'Justification')<br>
> TYPE:str LENGTH:? CHARACTER_SET: varies   

__The following columns are found only in SHIPMENTS sheet of DISP_Shipments_Cancellations__<br>
__RequisitionID__: (maps to 'Requisition ID' DISP_Shipments_Cancellation, sheet=SHIPMENTS) <br>
> TYPE:str LENGTH:? CHARACTER_SET: varies   

__The following columns are found only in CANCELLATIONS sheet of DISP_Shipments_Cancellations__<br>
__CancelledBy__: (maps to 'Cancelled By' DISP_Shipments_Cancellations, sheet=CANCELLATIONS only) <br>
> TYPE:str LENGTH: varies CHARACTER_SET: varies   

__RTDRef__: (maps to 'RTD Ref')<br>
> TYPE:str LENGTH: varies CHARACTER_SET: varies   

__ReasonCancelled__: (maps to 'Reason Cancelled')<br>
> TYPE:str LENGTH: varies CHARACTER_SET: varies   

__The following column added at the end for splitting the data into quarters__<br>
__Quarter__: created from 'RecordDate'<br>
> TYPE: str LENGTH: 6 CHARACTER_SET: [0-9Q]

## transfer_df created from DISP_AllStatesAndTerritories

#### Make sure you have set path_datafiles and LESOfile_all variables above.

In [4]:
trans_expected_columns = ['State', 'Station Name (LEA)',
                    'NSN', 'Item Name', 'Quantity', 'UI', 'Acquisition Value',
                    'DEMIL Code', 'DEMIL IC', 'Ship Date','Station Type']

trans_columns_dictionary = {'State':'StateAbbreviation', 'Station Name (LEA)':'RequestingAgency',
                      'NSN':'NSN', 'Item Name':'ItemDescription','Quantity':'Quantity',
                      'UI':'UnitIncrement', 'Acquisition Value':'AcquisitionValue',
                      'DEMIL Code':'DEMILCode','DEMIL IC':'DEMILIC',
                      'Ship Date':'RecordDate', 'Station Type':'StationType'}

In [5]:
# all sheets in the spreadsheet are read into a dictionary of dataframes
# see Check_DISP_AllStatesAndTerritoriesipynb for a full explanation
excel_dict = pd.read_excel("file:" + path_datafiles + LESOfile_all, sheet_name=None,
                           )

In [6]:
#CHECK What is the number of records in the original data?
total_transfers = 0
for k in excel_dict:
    total_transfers = total_transfers + len(excel_dict[k])
total_transfers

141068

In [7]:
# Convert the dictionary to a single dataframe with the records for all states/territories.
#    rename the columns to new schema (see 'Columns in Merged Dataframe' above)
#    strip leading/trailing white space from object types
transfer_df = pd.concat(
    [pd.concat([v],ignore_index=True) for k,v in excel_dict.items()],ignore_index=True).\
    rename(columns=trans_columns_dictionary).\
    apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

In [8]:
# Done with excel_dict, so release a bit of memory.
excel_dict.clear()

In [9]:
# Best guess for the AgencyType based on RequestingAgency; if no guess use 'other' as place holder.
transfer_df['AgencyType'] = transfer_df.RequestingAgency.map(get_agency_type)
transfer_df['AgencyType'].fillna('other',inplace=True)

In [10]:
# Break 'NSN' into NATO Stock Number units. (see 'Columns in Merged Dataframe' above)
transfer_df = transfer_df.assign(Item_FSG=transfer_df['NSN'].str.replace('-','').str[:2].values,
                   Item_FSC=transfer_df['NSN'].str.replace('-','').str[2:4].values,
                   Item_CC=transfer_df['NSN'].str.replace('-','').str[4:6].values,
                   Item_Code=transfer_df['NSN'].str.replace('-','').str[6:].values,)

In [11]:
# Fill missing columns with 'not in file' value to distinguish them from NaN/null values.
transfer_df['FSC'],transfer_df['NIIN'] = 'not in file','not in file'
transfer_df['Justification'] = 'not in file'
transfer_df['RequisitionID'],transfer_df['CancelledBy'] = 'not in file','not in file'
transfer_df['RTDRef'],transfer_df['ReasonCancelled'] = 'not in file','not in file'
# Fill 'OriginatingFile' column.
transfer_df['OriginatingFile'] = LESOfile_all + '[ALL]'
# TODO original sheet in file?

In [12]:
# Order the columns in preparation for merging.
transfer_df = transfer_df[ordered_columns_list]

###### Check Dataframe

In [13]:
transfer_df.shape
# Only DEMILIC should have NaN/null values; (see 'Columns in Merged Dataframe' above)
#transfer_df.isna().sum()

(141068, 24)

## shipments_df created from DISP_Shipments_Cancellations

#### Make sure you have set path_datafiles and LESOfile_qtr variables above.

In [14]:
ship_expected_columns = ['State', 'Station Name (LEA)', 'Requisition ID', 'FSC', 'NIIN',
                    'Item Name', 'UI', 'Quantity', 'Acquisition Value', 'Date Shipped',
                    'Justification']
ship_columns_dictionary = {'State':'StateAbbreviation', 'Station Name (LEA)':'RequestingAgency',
                      'Requisition ID':'RequisitionID', 'FSC':'FSC', 'NIIN':'NIIN',
                      'Item Name':'ItemDescription', 'UI':'UnitIncrement', 'Quantity':'Quantity',
                      'Acquisition Value':'AcquisitionValue', 'Date Shipped':'RecordDate', 
                      'Justification':'Justification'}

In [15]:
# Only 'SHIPMENTS' sheet in the original file is read into a dataframe.
# see Check_DISP_Shipments_Cancellations.ipynb for a full explanation
#    rename the columns to new schema (see 'Columns in Merged Dataframe' above)
#    strip leading/trailing white space from object types
shipments_df = pd.read_excel("file:" + path_datafiles + LESOfile_qtr, sheet_name='SHIPMENTS').\
                             rename(columns=ship_columns_dictionary).\
                             apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

In [16]:
#CHECK What is the number of records in the original data?
shipments_df.shape

(6353, 11)

In [17]:
# Best guess for the AgencyType based on RequestingAgency; if no guess use 'other' as place holder.
shipments_df['AgencyType'] = shipments_df.RequestingAgency.map(get_agency_type)
shipments_df['AgencyType'].fillna('other',inplace=True)

In [18]:
# Break 'FSC' and 'NIIN' into NATO Stock Number units. (see 'Columns in Merged Dataframe' above)
shipments_df = shipments_df.assign(Item_FSG=shipments_df['FSC'].astype(str).str[:2],
                   Item_FSC=shipments_df['FSC'].astype(str).str[2:4],
                   Item_CC=shipments_df['NIIN'].str[:2].values,
                   Item_Code=shipments_df['NIIN'].str[2:].values)

In [19]:
# Fill missing columns with 'not in file' value to distinguish them from NaN/null values.
shipments_df['NSN'],shipments_df['DEMILCode'] = 'not in file','not in file'
shipments_df['DEMILIC'],shipments_df['StationType'] = 'not in file','not in file'
shipments_df['CancelledBy'],shipments_df['RTDRef'] = 'not in file','not in file'
shipments_df['ReasonCancelled'] = 'not in file'
# Fill 'OriginatingFile' column.
shipments_df['OriginatingFile'] = LESOfile_qtr + '[SHIPMENTS]'

In [20]:
# Order the columns in preparation for merging.
shipments_df = shipments_df[ordered_columns_list]

###### Check Dataframe

In [21]:
shipments_df.shape
# Expect no NaN/null values; (see 'Columns in Merged Dataframe' above)
#shipments_df.isna().sum()

(6353, 24)

### Prepare cancellations_df From DISP_Shipments_Cancellations

#### Make sure you have set path_datafiles and LESOfile_qtr variables above.

In [22]:
canc_expected_columns = ['Cancelled By', 'RTD Ref', 'State', 'Station Name (LEA)',
                         'FSC', 'NIIN', 'Item Name', 'UI', 'Quantity', 'Acquisition Value',
                         'Date Requested', 'Justification', 'Reason Cancelled']
canc_columns_dictionary = {'Cancelled By':'CancelledBy', 'RTD Ref':'RTDRef', 
                           'State':'StateAbbreviation', 'Station Name (LEA)':'RequestingAgency',
                           'FSC':'FSC', 'NIIN':'NIIN', 'Item Name':'ItemDescription',
                           'UI':'UnitIncrement', 'Quantity':'Quantity', 'Acquisition Value':'AcquisitionValue',
                           'Date Requested':'RecordDate', 'Justification':'Justification',
                           'Reason Cancelled':'ReasonCancelled'}

In [23]:
# Only 'CANCELLATIONS' sheet in the original file is read into a dataframe.
# see Check_DISP_Shipments_Cancellations for a full explanation
#    rename the columns to new schema (see 'Columns in Merged Dataframe' above)
#    strip leading/trailing white space from object types
cancellations_df = pd.read_excel("file:" + path_datafiles + LESOfile_qtr, sheet_name='CANCELLATIONS').\
                             rename(columns=canc_columns_dictionary).\
                             apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

In [24]:
# Best guess for the AgencyType based on RequestingAgency; if no guess use 'other' as place holder.
cancellations_df['AgencyType'] = cancellations_df.RequestingAgency.map(get_agency_type)
cancellations_df['AgencyType'].fillna('other',inplace=True)

In [25]:
# Break 'FSC' and 'NIIN' into NATO Stock Number units. (see 'Columns in Merged Dataframe' above)
cancellations_df = cancellations_df.assign(Item_FSG=cancellations_df['FSC'].astype(str).str[:2],
                   Item_FSC=cancellations_df['FSC'].astype(str).str[2:4],
                   Item_CC=cancellations_df['NIIN'].str[:2].values,
                   Item_Code=cancellations_df['NIIN'].str[2:].values)

In [26]:
# Fill missing columns with 'not in file' value to distinguish them from NaN/null values.
cancellations_df['NSN'],cancellations_df['DEMILCode'] = 'not in file','not in file'
cancellations_df['DEMILIC'],cancellations_df['StationType'] = 'not in file','not in file'
cancellations_df['RequisitionID'] = 'not in file'
# Fill 'OriginatingFile' column.
cancellations_df['OriginatingFile'] = LESOfile_qtr + '[CANCELLATIONS]'

In [27]:
# Order the columns in preparation for merging.
cancellations_df = cancellations_df[ordered_columns_list]

In [28]:
cancellations_df.shape
# Found NaN/null values in 'Justification' and 'ReasonCancelled'; (see 'Columns in Merged Dataframe' above)
#cancellations_df.isna().sum()

(6640, 24)

### Merge All Datasets

In [29]:
if list(transfer_df.columns) != list(shipments_df.columns):
    print('Columns do not match.')
elif list(transfer_df.columns) != list(cancellations_df.columns):
    print('Columns do not match.')
elif list(shipments_df.columns) != list(cancellations_df.columns):
    print('Columns do not match.')

In [30]:
all_data_df = pd.concat([transfer_df,shipments_df,cancellations_df],axis=0)

### Thoughts on Storing the Merged Data

see https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d   

QUESTION: which columns are categorical?   
NOTE: none of these are evenly distributed   
OriginatingFile: categories will grow each month, so dates will change off these three files   
>DISP_AllStatesAndTerritories_mmddyyyy.xlsx[ALL]   
>DISP_Shipments_Territories_mmddyyyy_mmddyyyy.xlsx[SHIPMENTS]   
>DISP_Shipments_Territories_mmddyyyy_mmddyyyy.xlsx[CANCELLATIONS]   

StateAbbreviation: categorized by state; there are 59 possibilities based on   
>US Postal Service Publication 28 https://pe.usps.com/text/pub28/28apb.htm   
>(see Check_DISP_AllStatesAndTerritories.ipynb and Check_DISP_Shipments_Cancellations.ipynb   

Item_FSG: categorized by federal supply group; there are 100 possibilites, not evenly distributed   
>see https://en.wikipedia.org/wiki/Federal_Stock_Number#External_links   
>and https://en.wikipedia.org/wiki/List_of_NATO_Supply_Classification_Groups to start tracking these down   

AgencyType: rough guess of types of agencies based on station names, currently 13 categories   

TO EXPLORE: feather/parquet? should we save with more catgories?

In [ ]:
# If you want to save the entire dataset, this breaks the data into quarters
# and saves it in tab-separated files by quarter.
# BEWARE: so far no check on whether the data in files overlaps
all_data_df['Quarter'] = pd.PeriodIndex(all_data_df.RecordDate, freq='Q')                   

In [ ]:
# If the first time running it:
for i in list(all_data_df['Quarter'].unique()):
    #print(type(all_data_df[all_data_df['Quarter'] == i]))
    all_data_df[all_data_df['Quarter'] == i].to_csv('MergedData\\LESO_' + str(i) + '.tsv', 
                                                    index=False, mode='w', sep='\t', escapechar="\\") 

In [ ]:
# If appending to existing data:
for i in list(all_data_df['Quarter'].unique()):
    #print(type(all_data_df[all_data_df['Quarter'] == i]))
    all_data_df[all_data_df['Quarter'] == i].to_csv('MergedData\\LESO_' + str(i) + '.tsv', header=False,
                                                    index=False, mode='a', sep='\t', escapechar="\\") 

In [ ]:
# To read the data file:
#df = pd.read_csv('MergedData\\LESO_2020Q1.tsv',sep='\t',header=[0],index_col=None,
#                 quoting=csv.QUOTE_NONE, quotechar="",  escapechar="\\")

#df.columns
#df.shape